<a href="https://colab.research.google.com/github/THESHAMIK/Real-Life-Projects/blob/main/Pycaret/KHDS_Pycaret_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Here different options in SETUP is tried but all failed from base setup. So nORMALIZATION failed. Transformation = true failed. 
normalize = True, transformation = True, transform_target = True ==> All options tried individually still base setup is BEST.
BASE setup ==> exp_reg104 = setup(data = data_seen, target = 'Product Sulphur')

Then moved to hyper param tuning. Here OPTUNA won against all tunning options including base one. Next Optuna with additional params on BEST model performed better than plain normal OPTUNA.

# train dt using default hyperparameters
dt = create_model('dt') ==> Mean R2 is -1.5036
# tune hyperparameters with scikit-learn (default)
tuned_dt_sklearn = tune_model(dt) ==> Mean R2 is 0.0531 
# tune hyperparameters with scikit-optimize
tuned_dt_skopt = tune_model(dt, search_library = 'scikit-optimize') ==> Mean R2 is -0.0972
# tune hyperparameters with optuna
tuned_dt_optuna = tune_model(dt, search_library = 'optuna') ==> ==> Mean R2 is 
0.1203 ==> OPTUNA performed BEST among all hyper params from to -1.5 to 0.12.
So, OPTUNA should be used than default tune_model in pycaret.
# tune hyperparameters with tune-sklearn
tuned_dt_tuneskl = tune_model(dt, search_library = 'tune-sklearn')==> Mean R2 is ==> This module does not work with in basic form

For BEST model tunning best result with OPTUNA+options = R2= 0.1114.
This set of OPTUNA performed better than standard Optuna. Here iter = 50 from 10 + Early stopping + choose Better options added!

As a last step, Now let's take successful models from KHDS_Pycaret_6 which is top3 & blender of top3 & use Optuna hyper params to check output.
Simpler blender Mean R2 = 0.2189
Same blender now with Optuna hyper param --Mean R2 = 0.2645

In [ ]:
# Install pycaret library
!pip install pycaret

In [ ]:
from pycaret.utils import enable_colab
enable_colab()

Colab mode enabled.


In [ ]:
import pandas as pd
# Initiallize the data and fuctioning : Once initialize press 'Y'
from pycaret.regression import *

/usr/local/lib/python3.7/dist-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


In [ ]:
# Load the data
data=pd.read_csv('khds_final_1.csv')
data.head()

,LABfeed Sulphur,T1 : FRESH FEED FLOW TO KHDS KGPerHour,T1 : Reactor inlet temperature DEGC,T1 : Hydrogen make-up flow NM3perHour,T1 : Recycle hydrogen flow NM3perHour,T1 : REACTOR OUTLET Temp DEGC,T1 : REACTOR FEED INLET Pressure KGperCM2,T1 : Cold separator pressure KGperCM2,T1 : KERO FEED DENSITY KGperCM2,T2 : FRESH FEED FLOW TO KHDS KGPerHour,T2 : Reactor inlet temperature DEGC,T2 : Hydrogen make-up flow NM3perHour,T2 : Recycle hydrogen flow NM3perHour,T2 : REACTOR OUTLET Temp DEGC,T2 : REACTOR FEED INLET Pressure KGperCM2,T2 : Cold separator pressure KGperCM2,T2 : KERO FEED DENSITY KGperCM2,T3 : FRESH FEED FLOW TO KHDS KGPerHour,T3 : Reactor inlet temperature DEGC,T3 : Hydrogen make-up flow NM3perHour,T3 : Recycle hydrogen flow NM3perHour,T3 : REACTOR OUTLET Temp DEGC,T3 : REACTOR FEED INLET Pressure KGperCM2,T3 : Cold separator pressure KGperCM2,T3 : KERO FEED DENSITY KGperCM2,T4 : FRESH FEED FLOW TO KHDS KGPerHour,T4 : Reactor inlet temperature DEGC,T4 : Hydrogen make-up flow NM3perHour,T4 : Recycle hydrogen flow NM3perHour,T4 : REACTOR OUTLET Temp DEGC,T4 : REACTOR FEED INLET Pressure KGperCM2,T4 : Cold separator pressure KGperCM2,T4 : KERO FEED DENSITY KGperCM2,Product Sulphur
0,1000.0,72482.43948,288.485914,685.146132,11114.58673,288.637635,21.927602,18.775248,0.0,72337.37151,287.651367,740.221182,11041.21585,288.761036,21.810156,18.688527,0.0,72376.86708,287.823835,660.058960,11072.44088,287.738775,21.813427,18.693064,0.0,72353.77432,287.978641,614.163076,11226.93106,288.712160,21.805388,18.666758,0.0,2
1,1400.0,75588.69117,286.406314,571.248477,11259.14433,286.921575,21.911739,18.702087,0.0,75746.39266,286.441067,549.330129,11228.98135,286.630241,21.870260,18.674408,0.0,75963.37102,286.687275,617.336266,11004.25626,286.128780,21.711249,18.533493,0.0,75918.19844,287.134693,464.921691,11066.97566,286.615329,21.934672,18.747208,0.0,0.5
2,1300.0,75675.29792,284.035734,564.716667,10668.45313,284.140627,21.900282,18.734029,0.0,75628.86875,284.037801,582.175509,10721.20293,284.517944,21.882274,18.724772,0.0,75646.63073,283.697494,559.376522,10763.02799,284.325244,21.813410,18.677979,0.0,76002.57969,284.800492,580.681543,10701.71289,284.840621,21.848295,18.704427,0.0,3
3,1500.0,46269.02109,276.210773,371.162782,14497.57829,276.808471,21.747162,19.528809,0.0,46001.59297,276.116185,527.354706,14646.65260,277.295914,21.531470,19.330088,0.0,46411.94089,275.763115,258.167509,14658.63428,277.148141,21.664635,19.450734,0.0,46176.64115,275.822623,565.165267,14783.88027,276.540142,21.547435,19.357554,0.0,4
4,1400.0,45394.43555,276.126803,296.835800,14606.11231,276.776767,21.560785,19.492783,0.0,45231.54465,276.010997,441.971037,14684.18629,276.807028,21.579045,19.508255,0.0,45301.01027,276.366341,365.445542,14693.11828,276.877170,21.526057,19.478851,0.0,45265.78914,276.090728,409.074912,14675.20297,277.132889,21.597432,19.496689,0.0,3


In [ ]:
#U can see last column Product Sulpur(Y) is an object & not Float
#data['Product Sulphur']=data['Product Sulphur'].astype(float64) ##==> # This is NOT working
data['Product Sulphur'] = pd.to_numeric(data['Product Sulphur'],errors = 'coerce') ##==> This one works as pd.to_numeric works for convert a list, a series, an array, or a tuple to a numeric datatype
#type(data['Product Sulphur']) ##This shows that Product Sulphur data type is a series hence astype did not work
data.dtypes
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 711 entries, 0 to 710
Data columns (total 34 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   LABfeed Sulphur                            711 non-null    float64
 1   T1 : FRESH FEED FLOW TO KHDS KGPerHour     711 non-null    float64
 2   T1 : Reactor inlet temperature DEGC        711 non-null    float64
 3   T1 : Hydrogen make-up flow NM3perHour      711 non-null    float64
 4   T1 : Recycle hydrogen flow NM3perHour      711 non-null    float64
 5   T1 : REACTOR OUTLET Temp DEGC              711 non-null    float64
 6   T1 : REACTOR FEED INLET Pressure KGperCM2  711 non-null    float64
 7   T1 : Cold separator pressure KGperCM2      711 non-null    float64
 8   T1 : KERO FEED DENSITY KGperCM2            711 non-null    float64
 9   T2 : FRESH FEED FLOW TO KHDS KGPerHour     711 non-null    float64
 10  T2 : Reactor inlet tempera

In [ ]:
#data_seen = data.sample(frac=0.5, random_state=786)
data_seen = data.sample(frac=0.9)
#data_unseen = data.drop(data.index) ==> Dangerous statement making unseen data being BLANK
data_unseen = data.sample(frac=0.1)
#data_seen.reset_index(drop=True, inplace=True)
#data_unseen.reset_index(drop=True, inplace=True)
print('Data for Modeling: ' + str(data_seen.shape))
print('Unseen Data For Predictions: ' + str(data_unseen.shape))

Data for Modeling: (640, 34)
Unseen Data For Predictions: (71, 34)


In [ ]:
data_unseen.reset_index(drop=True, inplace=True)

In [ ]:
data_seen.reset_index(drop=True, inplace=True)

In [ ]:
from pycaret.regression import *
#exp_reg102 = setup(data = data_seen, target = 'Product Sulphur', session_id=999, normalize = True, transformation = True, transform_target = True)

#exp_reg103 = setup(data = data_seen, target = 'Product Sulphur', session_id=999, transform_target = True)
##All options failed so lets stick to basic setup
#exp_reg104 = setup(data = data_seen, target = 'Product Sulphur', session_id=123, remove_outliers = True)
exp_reg104 = setup(data = data_seen, target = 'Product Sulphur')

,Description,Value
0,session_id,4893
1,Target,Product Sulphur
2,Original Data,"(640, 34)"
3,Missing Values,True
4,Numeric Features,33
5,Categorical Features,0
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(446, 12)"


In [ ]:
# train all models using default hyperparameters
best = compare_models()

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
et,Extra Trees Regressor,1.8933,24.2672,3.7914,0.2145,0.4369,0.5441,0.446
huber,Huber Regressor,1.7955,25.6453,4.0246,0.0941,0.4378,0.4217,0.042
br,Bayesian Ridge,2.0152,25.4686,4.0625,0.0434,0.4733,0.6144,0.014
omp,Orthogonal Matching Pursuit,2.0490,25.8127,4.1027,0.0266,0.4891,0.6498,0.013
lasso,Lasso Regression,2.0472,25.7277,4.1049,-0.0006,0.4799,0.6091,0.017
en,Elastic Net,2.0507,25.7514,4.1091,-0.0044,0.4814,0.6103,0.016
lightgbm,Light Gradient Boosting Machine,2.1566,26.2406,4.1296,-0.0238,0.5109,0.6555,0.087
dummy,Dummy Regressor,2.2180,26.1818,4.1862,-0.0528,0.5241,0.7693,0.011
llar,Lasso Least Angle Regression,2.2180,26.1818,4.1862,-0.0528,0.5241,0.7693,0.013
knn,K Neighbors Regressor,2.1491,28.7334,4.4355,-0.2514,0.4834,0.5717,0.062


In [ ]:
tuned_best_model = tune_model(best)


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,1.0808,2.8744,1.6954,0.4003,0.3152,0.3624
1,3.1119,143.5484,11.9812,-0.0201,0.5458,0.3817
2,1.6766,10.1916,3.1924,0.0289,0.3909,0.3898
3,1.7228,5.5042,2.3461,0.4432,0.4201,0.6371
4,1.1081,2.5619,1.6006,0.4278,0.3205,0.4116
5,1.5489,7.2026,2.6838,0.2789,0.4020,0.5259
6,1.4959,3.7087,1.9258,0.1228,0.3814,0.5100
7,1.2074,2.4706,1.5718,0.4459,0.3394,0.4933
8,1.9667,17.4094,4.1725,0.1471,0.4504,0.4656


In [ ]:
!pip install tune-sklearn ray[tune]

In [ ]:
!pip install hyperopt

In [ ]:
#tuned_best_model_hyperopt = tune_model(best,search_library="tune-sklearn",search_algorithm="hyperopt",n_iter=10)

IntProgress(value=0, description='Processing: ', max=7)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,09:46:05
Status,. . . . . . . . . . . . . . . . . .,Searching Hyperparameters
Estimator,. . . . . . . . . . . . . . . . . .,Random Forest Regressor


,Fold,MAE,MSE,RMSE,R2,RMSLE,MAPE


ValueError: ignored

In [ ]:
!pip install optuna

In [ ]:
from tune_sklearn import TuneGridSearchCV
tuned_best_model_optuna = tune_model(best, search_library = 'tune-sklearn')

IntProgress(value=0, description='Processing: ', max=7)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,09:46:46
Status,. . . . . . . . . . . . . . . . . .,Searching Hyperparameters
Estimator,. . . . . . . . . . . . . . . . . .,Random Forest Regressor


,Fold,MAE,MSE,RMSE,R2,RMSLE,MAPE


2022-04-21 09:46:46,384	INFO tune.py:837 -- Initializing Ray automatically.For cluster usage or custom Ray initialization, call `ray.init(...)` before `tune.run`.


ValueError: ignored

In [ ]:
ada = create_model('ada')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,2.7060,18.3219,4.2804,-0.4845,0.6223,1.1258
1,2.9217,14.0013,3.7418,-1.6939,0.6134,1.1046
2,2.1322,6.2371,2.4974,-0.2954,0.6000,1.2425
3,2.6185,12.5876,3.5479,-2.5542,0.6416,1.2878
4,2.4348,15.8162,3.9770,-0.5987,0.5640,0.9113
5,2.2290,8.2116,2.8656,-0.3167,0.5195,0.9061
6,3.3895,19.2932,4.3924,-0.3636,0.6802,1.2681
7,4.0260,151.8416,12.3224,-0.0182,0.6426,0.7829
8,2.5255,14.8602,3.8549,0.1111,0.5421,0.9045


In [ ]:
tuned_top1 = tune_model(ada,
                        optimize="R2",
                        search_library="tune-sklearn",
                        search_algorithm="hyperopt",
                        choose_better = True ,
                        early_stopping = "asha",
                        early_stopping_max_iters = 10,
                        return_tuner = False)

IntProgress(value=0, description='Processing: ', max=7)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,09:46:59
Status,. . . . . . . . . . . . . . . . . .,Searching Hyperparameters
Estimator,. . . . . . . . . . . . . . . . . .,AdaBoost Regressor


,Fold,MAE,MSE,RMSE,R2,RMSLE,MAPE


2022-04-21 09:46:59,535	INFO tune.py:837 -- Initializing Ray automatically.For cluster usage or custom Ray initialization, call `ray.init(...)` before `tune.run`.


ValueError: ignored

In [ ]:
import optuna
tuned_model, tuner = tune_model(ada,
                         n_iter=50,
                         optimize='R2',
                         search_library='optuna',
                         search_algorithm='tpe',
                         early_stopping='asha',
                         choose_better=False,
                         verbose=True,
                         tuner_verbose=4,
                         return_tuner=True
                        )

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,1.6488,11.5005,3.3912,0.0682,0.4130,0.5140
1,1.5405,3.7673,1.9410,0.2752,0.3587,0.5041
2,1.5153,3.9953,1.9988,0.1702,0.4448,0.7110
3,1.3250,3.0751,1.7536,0.1317,0.3637,0.4957
4,1.6233,8.5855,2.9301,0.1322,0.3959,0.4407
5,1.4344,4.2233,2.0551,0.3228,0.3569,0.4494
6,2.1327,13.6962,3.7008,0.0320,0.5059,0.6563
7,4.1727,156.5161,12.5106,-0.0495,0.6778,0.7856
8,1.8228,15.1166,3.8880,0.0957,0.4329,0.4961


In [ ]:
import optuna
tuned_model, tuner = tune_model(best,
                         n_iter=50,
                         optimize='R2',
                         search_library='optuna',
                         search_algorithm='tpe',
                         early_stopping='asha',
                         choose_better=False,
                         verbose=True,
                         tuner_verbose=4,
                         return_tuner=True
                        )

IntProgress(value=0, description='Processing: ', max=7)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,10:11:22
Status,. . . . . . . . . . . . . . . . . .,Searching Hyperparameters
Estimator,. . . . . . . . . . . . . . . . . .,Random Forest Regressor


,Fold,MAE,MSE,RMSE,R2,RMSLE,MAPE


[I 2022-04-21 10:11:22,956] Searching the best hyperparameters using 447 samples...
exception calling callback for <Future at 0x7f7ffb92f050 state=finished raised ShutdownExecutorError>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/joblib/externals/loky/_base.py", line 625, in _invoke_callbacks
    callback(self)
  File "/usr/local/lib/python3.7/dist-packages/joblib/parallel.py", line 359, in __call__
    self.parallel.dispatch_next()
  File "/usr/local/lib/python3.7/dist-packages/joblib/parallel.py", line 792, in dispatch_next
    if not self.dispatch_one_batch(self._original_iterator):
  File "/usr/local/lib/python3.7/dist-packages/joblib/parallel.py", line 859, in dispatch_one_batch
    self._dispatch(tasks)
  File "/usr/local/lib/python3.7/dist-packages/joblib/parallel.py", line 777, in _dispatch
    job = self._backend.apply_async(batch, callback=cb)
  File "/usr/local/lib/python3.7/dist-packages/joblib/_parallel_backends.py", line 531, in apply

KeyboardInterrupt: ignored

In [ ]:
#tune-sklearn - tune-sklearn ray[tune]
import optuna
from tune_sklearn import TuneGridSearchCV
#import tune-sklearn ray[tune]
tuned_model, tuner = tune_model(best,
                         n_iter=50,
                         optimize='R2',
                         search_library='tune-sklearn',
                         search_algorithm='grid',
                         early_stopping='asha',
                         choose_better=False,
                         verbose=True,
                         tuner_verbose=4,
                         return_tuner=True
                        )


In [ ]:
dt = create_model('dt')

In [ ]:
tuned_dt_sklearn = tune_model(dt)

In [ ]:
tuned_dt_optuna = tune_model(dt, search_library = 'optuna')

In [ ]:
tuned_dt_tuneskl = tune_model(dt, search_library = 'tune-sklearn')

In [ ]:
!pip install scikit-optimize

     |████████████████████████████████| 100 kB 3.9 MB/s 


In [ ]:
tuned_dt_skopt = tune_model(dt, search_library = 'scikit-optimize')

# train dt using default hyperparameters
dt = create_model('dt') ==> Mean R2 is -1.5036
# tune hyperparameters with scikit-learn (default)
tuned_dt_sklearn = tune_model(dt) ==> Mean R2 is 0.0531 
# tune hyperparameters with scikit-optimize
tuned_dt_skopt = tune_model(dt, search_library = 'scikit-optimize') ==> Mean R2 is -0.0972
# tune hyperparameters with optuna
tuned_dt_optuna = tune_model(dt, search_library = 'optuna') ==> ==> Mean R2 is 
0.1203 ==> OPTUNA performed BEST among all hyper params from to -1.5 to 0.12.
So, OPTUNA should be used than default tune_model in pycaret.
# tune hyperparameters with tune-sklearn
tuned_dt_tuneskl = tune_model(dt, search_library = 'tune-sklearn')==> Mean R2 is ==> This module does not work with in basic form

In [ ]:
tuned_dt_sklearn = tune_model(best) 

In [ ]:
tuned_dt_skopt = tune_model(best, search_library = 'scikit-optimize')

In [ ]:
tuned_dt_optuna = tune_model(best, search_library = 'optuna')

In [ ]:
import optuna
tuned_model, tuner = tune_model(best,
                         n_iter=50,
                         optimize='R2',
                         search_library='optuna',
                         search_algorithm='tpe',
                         early_stopping='asha',
                         choose_better=False,
                         verbose=True,
                         tuner_verbose=4,
                         return_tuner=True
                        )

For BEST model tunning best result with OPTUNA+options = R2= 0.1114.
This set of OPTUNA performed better than standard Optuna. Here iter = 50 from 10 + Early stopping + choose Better options added!

In [ ]:
tuned_dt_optuna = tune_model(best, search_library = 'optuna')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,1.2637,3.2446,1.8013,0.3230,0.3461,0.4477
1,3.2260,144.4492,12.0187,-0.0265,0.5584,0.4335
2,1.7575,10.0817,3.1752,0.0394,0.4065,0.4595
3,1.9185,5.3412,2.3111,0.4597,0.4823,0.8539
4,1.1596,2.2448,1.4983,0.4986,0.3230,0.4525
5,1.6897,6.6692,2.5825,0.3323,0.4416,0.6695
6,1.6704,4.6912,2.1659,-0.1095,0.4458,0.6559
7,1.1480,2.1807,1.4767,0.5109,0.3437,0.5214
8,2.5330,19.0445,4.3640,0.0670,0.5079,0.6650


Now let's take successful models from KHDS_Pycaret_6 which is top3 & blender of top3 & use Optuna hyper params to check output.

In [ ]:
top3 = compare_models(n_select = 3)


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
et,Extra Trees Regressor,1.8933,24.2672,3.7914,0.2145,0.4369,0.5441,0.450
huber,Huber Regressor,1.7955,25.6453,4.0246,0.0941,0.4378,0.4217,0.044
br,Bayesian Ridge,2.0152,25.4686,4.0625,0.0434,0.4733,0.6144,0.013
omp,Orthogonal Matching Pursuit,2.0490,25.8127,4.1027,0.0266,0.4891,0.6498,0.013
lasso,Lasso Regression,2.0472,25.7277,4.1049,-0.0006,0.4799,0.6091,0.017
en,Elastic Net,2.0507,25.7514,4.1091,-0.0044,0.4814,0.6103,0.016
lightgbm,Light Gradient Boosting Machine,2.1566,26.2406,4.1296,-0.0238,0.5109,0.6555,0.052
dummy,Dummy Regressor,2.2180,26.1818,4.1862,-0.0528,0.5241,0.7693,0.011
llar,Lasso Least Angle Regression,2.2180,26.1818,4.1862,-0.0528,0.5241,0.7693,0.013
knn,K Neighbors Regressor,2.1491,28.7334,4.4355,-0.2514,0.4834,0.5717,0.063


In [ ]:
import optuna
tuned_model, tuner = tune_model(top3,
                         n_iter=50,
                         optimize='R2',
                         search_library='optuna',
                         search_algorithm='tpe',
                         early_stopping='asha',
                         choose_better=False,
                         verbose=True,
                         tuner_verbose=4,
                         return_tuner=True
                        )

ValueError: ignored

In [ ]:
blender = blend_models(top3)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,1.3185,3.1390,1.7717,0.3451,0.3458,0.4767
1,2.9618,141.7371,11.9053,-0.0073,0.5202,0.3340
2,1.6993,9.7555,3.1234,0.0705,0.3963,0.4419
3,1.7269,5.3563,2.3144,0.4581,0.4228,0.6622
4,1.1274,2.2642,1.5047,0.4943,0.3185,0.4390
5,1.6699,7.5382,2.7456,0.2453,0.4241,0.5969
6,1.4959,3.7559,1.9380,0.1117,0.3846,0.5114
7,1.1286,2.2976,1.5158,0.4847,0.3396,0.5059
8,2.1285,19.3396,4.3977,0.0526,0.4766,0.5414


In [ ]:
import optuna
tuned_model, tuner = tune_model(blender,
                         n_iter=50,
                         optimize='R2',
                         search_library='optuna',
                         search_algorithm='tpe',
                         early_stopping='asha',
                         choose_better=False,
                         verbose=True,
                         tuner_verbose=4,
                         return_tuner=True
                        )

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,1.2852,3.1906,1.7862,0.3343,0.3356,0.4285
1,3.0298,142.4685,11.9360,-0.0125,0.5316,0.3491
2,1.6464,9.7973,3.1301,0.0665,0.3900,0.4141
3,1.5754,4.0953,2.0237,0.5857,0.4035,0.6356
4,0.9540,1.5769,1.2557,0.6478,0.2803,0.3740
5,1.5756,5.9482,2.4389,0.4045,0.4032,0.5705
6,1.4852,3.7177,1.9281,0.1207,0.3833,0.5025
7,1.1615,2.3771,1.5418,0.4669,0.3297,0.4739
8,2.2527,18.4757,4.2983,0.0949,0.4852,0.5602


In [ ]:
top3_boosted = ensemble_model(top3, method = 'Boosting')

ValueError: ignored

In [ ]:
best_boosted = ensemble_model(best, method = 'Boosting')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,1.5282,4.4356,2.1061,0.0746,0.4055,0.5572
1,3.3907,142.9229,11.9550,-0.0157,0.5672,0.5111
2,1.8752,11.2418,3.3529,-0.0711,0.4429,0.5302
3,1.8949,5.2887,2.2997,0.4650,0.5012,0.9060
4,1.1833,2.2762,1.5087,0.4916,0.3483,0.5087
5,2.0166,7.8769,2.8066,0.2114,0.5318,0.9315
6,1.7969,5.3236,2.3073,-0.2591,0.4595,0.6842
7,1.4645,3.5670,1.8887,0.2000,0.4198,0.6828
8,2.6780,21.2407,4.6088,-0.0405,0.5351,0.7108


In [ ]:
blender_boosted = ensemble_model(blender, method = 'Boosting')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,3.3589,19.0061,4.3596,-2.9654,0.6729,1.3439
1,3.9168,142.2040,11.9249,-0.0106,0.5910,0.6665
2,4.0137,26.2072,5.1193,-1.4971,0.7170,1.3245
3,1.7736,5.2152,2.2837,0.4724,0.4607,0.7817
4,3.9239,24.5103,4.9508,-4.4744,0.6827,1.4624
5,8.9509,125.3553,11.1962,-11.5498,1.1868,3.3069
6,4.2270,27.0556,5.2015,-5.3990,0.7232,1.5274
7,3.6920,25.1464,5.0146,-4.6399,0.6424,1.4428
8,5.1328,48.6977,6.9784,-1.3856,0.8117,1.4906


In [ ]:

blender_tuned_boosted = ensemble_model(tuned_model, method = 'Boosting')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,4.4053,45.5541,6.7494,-8.5044,0.7972,1.8323
1,3.6541,139.9716,11.8310,0.0053,0.5604,0.5983
2,6.9859,84.6222,9.1990,-7.0629,0.9275,1.9444
3,7.2048,93.7899,9.6845,-8.4881,0.9976,2.6599
4,4.4768,38.8663,6.2343,-7.6808,0.7795,1.5179
5,3.3728,18.2377,4.2706,-0.8259,0.6984,1.3983
6,4.5867,39.9549,6.3210,-8.4498,0.7663,1.4949
7,3.6133,25.8168,5.0810,-4.7902,0.6850,1.4060
8,5.7034,66.9019,8.1794,-2.2774,0.8569,1.6415
